In [1]:
# 创建SparkSession
#################################
%spark --conf spark.yarn.queue=root.zw03.hadoop-caterb2b.etl \
 --conf spark.app.name=Promodel_tests12 \
 --conf spark.num.executors=80 \
 --conf spark.files.overwrite=true \
 --conf spark.executor.cores=4 \
 --conf spark.executor.memory=8G \
 --conf spark.driver.memory=16G \
 --conf spark.yarn.executor.memoryOverhead=16G

,Driver日志,队列信息
,/home/sankuai/logs/spark-1634176827.log,root.zw03.hadoop-caterb2b.etl


CmdOutput(['tail', '-50', '/home/sankuai/logs/spark-1634176827.log'], 2)

,SparkSession,SparkContext,Job Search/applicationId
链接/变量名,spark,sc,application_1632474272105_928563


In [2]:
# 导入工具包
########################################################################
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql import Window
import pyspark.sql.functions as F

import warnings
warnings.filterwarnings("ignore")

import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

from itertools import chain
import pickle
from sklearn.preprocessing import LabelEncoder
import multiprocessing

import random
random.seed(42)
np.random.seed(42)

In [3]:
sc.addPyFile("promodel_feature_util.py")

from promodel_feature_util import FeatureUtil

In [4]:
# 日期参数设定
forecast_date = '20210826'
# forecast_date = datetime.now().strftime('%Y%m%d')
train_start_date = (pd.to_datetime(forecast_date) - timedelta(182)).strftime('%Y%m%d')      ## 训练集开始日期
train_end_date = (pd.to_datetime(forecast_date) - timedelta(1)).strftime('%Y%m%d')         ## 训练集截止日期
start_date = (pd.to_datetime(train_end_date) - timedelta(31)).strftime('%Y%m%d')          ## 筛选case的开始日期
forecast_end_date = (pd.to_datetime(forecast_date) + timedelta(6)).strftime('%Y%m%d')     ## 测试集截止日期

In [5]:
print("start: {}, end: {}, \ntrain_start_date: {}, \nforecast_end_date: {}".format(start_date, train_end_date, train_start_date, forecast_end_date))
print("Read Hive Data...")
start_time = time.time()

start: 20210725, end: 20210825, 
train_start_date: 20210225, 
forecast_end_date: 20210901
Read Hive Data...


In [6]:
train_sql = """
        SELECT n.*
          FROM (
                SELECT *
                  FROM (
                        SELECT *,
                               avg(arranged_cnt) over(partition by wh_id, sku_id order by dt rows between 2 preceding and 0 preceding) as last_3days_mean,
                               avg(arranged_cnt) over(partition by wh_id, sku_id order by dt rows between 9 preceding and 3 preceding) as fore_7days_mean,
                               avg(ape_T0) over(partition by wh_id, sku_id order by dt rows between 2 preceding and 0 preceding) as last_3days_mean_apeT0,
                               avg(ape_T0) over(partition by wh_id, sku_id order by dt rows between 9 preceding and 3 preceding) as fore_7days_mean_apeT0,
                               COUNT(sku_id) OVER (PARTITION BY wh_id, sku_id) as row_num,
                               ROW_NUMBER() OVER (PARTITION BY wh_id, sku_id ORDER BY dt desc) as latest
                          FROM (
                                SELECT i.*,
                                       j.pred_t0,
                                       case when (i.arranged_cnt = 0 and j.pred_t0 >= 1) then j.pred_t0
                                            when (i.arranged_cnt = 0 and j.pred_t0 < 1)  then 0
                                            else abs(j.pred_t0 - i.arranged_cnt) / i.arranged_cnt
                                             end as ape_T0
                                  FROM (
                                        select *
                                          from mart_caterb2b_forecast.app_caterb2b_forecast_sale_input_v3
                                         where dt BETWEEN {} and {}
                                           and is_train = 1
                                       ) i
                                  join (
                                        select *
                                          from mart_caterb2b_forecast.app_forecast_sale_badcase_general_analysis
                                         where dt BETWEEN {} and {}
                                       ) j
                                    on i.dt = j.dt
                                   and i.bu_id = j.bu_id
                                   and i.wh_id = j.wh_id
                                   and i.sku_id = j.sku_id
                               )
                       )
                 where ape_T0 is not NULL
                   and row_num > 10
                   and latest = 1
                   and last_3days_mean_apeT0 >= 0.5
                   and 0.5 * (fore_7days_mean+1) > (last_3days_mean+1)
               ) m
          join (
                SELECT *
                  FROM (
                        SELECT *,
                               COUNT(sku_id) OVER (PARTITION BY wh_id, sku_id) as row_num
                          FROM mart_caterb2b_forecast.app_caterb2b_forecast_sale_input_v3
                         where dt BETWEEN {} and {}
                           and is_train = 1
                       ) da
                 where da.row_num >= 60
               ) n
            on m.bu_id = n.bu_id
           and m.wh_id = n.wh_id
           and m.sku_id = n.sku_id
        """

test_sql = """
        SELECT n.*
          FROM (
                SELECT *
                  FROM (
                        SELECT *,
                               avg(arranged_cnt) over(partition by wh_id, sku_id order by dt rows between 2 preceding and 0 preceding) as last_3days_mean,
                               avg(arranged_cnt) over(partition by wh_id, sku_id order by dt rows between 9 preceding and 3 preceding) as fore_7days_mean,
                               avg(ape_T0) over(partition by wh_id, sku_id order by dt rows between 2 preceding and 0 preceding) as last_3days_mean_apeT0,
                               avg(ape_T0) over(partition by wh_id, sku_id order by dt rows between 9 preceding and 3 preceding) as fore_7days_mean_apeT0,
                               COUNT(sku_id) OVER (PARTITION BY wh_id, sku_id) as row_num,
                               ROW_NUMBER() OVER (PARTITION BY wh_id, sku_id ORDER BY dt desc) as latest
                          FROM (
                                SELECT i.*,
                                       j.pred_t0,
                                       case when (i.arranged_cnt = 0 and j.pred_t0 >= 1) then j.pred_t0
                                            when (i.arranged_cnt = 0 and j.pred_t0 < 1)  then 0
                                            else abs(j.pred_t0 - i.arranged_cnt) / i.arranged_cnt
                                             end as ape_T0
                                  FROM (
                                        select *
                                          from mart_caterb2b_forecast.app_caterb2b_forecast_sale_input_v3
                                         where dt BETWEEN {} and {}
                                           and is_train = 1
                                       ) i
                                  join (
                                        select *
                                          from mart_caterb2b_forecast.app_forecast_sale_badcase_general_analysis
                                         where dt BETWEEN {} and {}
                                       ) j
                                    on i.dt = j.dt
                                   and i.bu_id = j.bu_id
                                   and i.wh_id = j.wh_id
                                   and i.sku_id = j.sku_id
                               )
                       )
                 where ape_T0 is not NULL
                   and row_num > 10
                   and latest = 1
                   and last_3days_mean_apeT0 >= 0.5
                   and 0.5 * (fore_7days_mean+1) > (last_3days_mean+1)
               ) m
          join (
                SELECT *
                  FROM mart_caterb2b_forecast.app_caterb2b_forecast_sale_input_v3
                 where dt BETWEEN {} and {}
                   and is_train = 0
               ) n
            on m.bu_id = n.bu_id
           and m.wh_id = n.wh_id
           and m.sku_id = n.sku_id
        """


train_data = spark.sql(train_sql.format(start_date, train_end_date, 
                                        start_date, train_end_date,
                                        train_start_date, train_end_date))

test_data = spark.sql(test_sql.format(start_date, train_end_date, 
                                      start_date, train_end_date,
                                      forecast_date, forecast_end_date))

configs = {}
## 更新配置参数字典
append_conf = {"forecast_date": forecast_date,
               "start_date": start_date,
               "train_start_date": train_start_date,
               "train_end_date": train_end_date,
               "forecast_end_date": forecast_end_date
               }
configs.update(append_conf)

In [7]:
train_data.count(), test_data.count()

(491392, 25648)

In [8]:
all_data = train_data.drop('row_num').union(test_data)

In [9]:
all_data.count()

517040

In [10]:
all_data.printSchema()

root
 |-- is_train: integer (nullable = true)
 |-- bu_id: long (nullable = true)
 |-- wh_id: long (nullable = true)
 |-- sku_id: long (nullable = true)
 |-- cat1_id: long (nullable = true)
 |-- cat1_name: string (nullable = true)
 |-- cat2_name: string (nullable = true)
 |-- two_embed0: double (nullable = true)
 |-- two_embed1: double (nullable = true)
 |-- cnt_band: integer (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- tax_rate: integer (nullable = true)
 |-- csu_origin_price: double (nullable = true)
 |-- w_price: double (nullable = true)
 |-- seq_num: integer (nullable = true)
 |-- seq_price: double (nullable = true)
 |-- redu_num: integer (nullable = true)
 |-- csu_redu_num: integer (nullable = true)
 |-- csu_redu_mdct: double (nullable = true)
 |-- csu_redu_adct: double (nullable = true)
 |-- cir_redu_num: integer (nullable = true)
 |-- cir_redu_mdct: double (nullable = true)
 |-- cir_redu_adct: double (nullable = true)
 |-- pro_num: integer (nullable = true)
 |

In [11]:
cols = ['bu_id', 'wh_id', 'sku_id', 'cat1_name', 'cat2_name',
        'seq_num', 'redu_num', 'csu_redu_num', 'cir_redu_num', 'pro_num',
        'day_abbr', 'is_work_day', 'is_weekend', 'is_holiday',
        'is_train', 'arranged_cnt', 'dt']

all_data = all_data.select(*cols)

In [12]:
num_cols = ['seq_num', 'redu_num', 'csu_redu_num', 'cir_redu_num', 'pro_num']
all_data = all_data.fillna(0, subset=num_cols)
all_data = all_data.withColumn('all_pro_num', (all_data['seq_num'] + all_data['redu_num'] + all_data['pro_num']))
all_data = all_data.sort(F.col('bu_id'), F.col('wh_id'), F.col('sku_id'), F.col('dt'))

In [13]:
all_data.show()

+------+-----+--------+---------+---------+-------+--------+------------+------------+-------+--------+-----------+----------+----------+--------+------------+--------+-----------+
| bu_id|wh_id|  sku_id|cat1_name|cat2_name|seq_num|redu_num|csu_redu_num|cir_redu_num|pro_num|day_abbr|is_work_day|is_weekend|is_holiday|is_train|arranged_cnt|      dt|all_pro_num|
+------+-----+--------+---------+---------+-------+--------+------------+------------+-------+--------+-----------+----------+----------+--------+------------+--------+-----------+
|110100|  159|10000124| 肉禽水产（冷冻）|      鸡肉冻|      0|       0|           0|           0|      1|     Thu|          1|         0|         0|       0|           0|20210826|          1|
|110100|  159|10000124| 肉禽水产（冷冻）|      鸡肉冻|      0|       0|           0|           0|      1|     Fri|          1|         0|         0|       0|           0|20210827|          1|
|110100|  159|10000124| 肉禽水产（冷冻）|      鸡肉冻|      0|       0|           0|           0|      1| 

In [14]:
## 更新配置参数字典
append_conf = {"bd_all_data_col_names": all_data.schema.names
               }
configs.update(append_conf)

In [15]:
def seqOp(x, y):
    '''seqFunc'''
    x.append(y)
    return x

def combOp(x, y):
    '''combFunc'''
    return x+y

In [16]:
# 生产销量特征
###############################################################################

def arr_feature(all_df, configs):
    sku_data = pd.DataFrame(all_df[1], columns=configs['bd_all_data_col_names']) \
                 .replace('', np.nan)
    
    if len(sku_data[sku_data.is_train==1]) < 60:
        return []
    
    wh_id,sku_id = int(sku_data.wh_id.unique()[0]), int(sku_data.sku_id.unique()[0])
    sku_data = sku_data.sort_values('dt')
    feature_obj = FeatureUtil()
    
    arr_list = sku_data[sku_data.is_train == 1].arranged_cnt
    roll_7d_avg = arr_list.rolling(30, min_periods=30).apply(feature_obj.clc_fc_val).dropna()

    avg_7d_0 = arr_list.rolling(30, min_periods=30).apply(lambda x: feature_obj.window_avg_arr(x, 0)).dropna()
    avg_7d_1 = arr_list.rolling(30, min_periods=30).apply(lambda x: feature_obj.window_avg_arr(x, 1)).dropna()
    avg_7d_2 = arr_list.rolling(30, min_periods=30).apply(lambda x: feature_obj.window_avg_arr(x, 2)).dropna()
    avg_7d_3 = arr_list.rolling(30, min_periods=30).apply(lambda x: feature_obj.window_avg_arr(x, 3)).dropna()

    avg_3d_0 = arr_list.rolling(30, min_periods=30).apply(lambda x: feature_obj.window_avg_arr(x, 4)).dropna()
    avg_last7d = arr_list.rolling(30, min_periods=30).apply(lambda x: feature_obj.latest_val(x)).dropna()
    
    # 外推
    arr_and_avg  = roll_7d_avg.tolist()
    arr_and_avg0 = avg_7d_0.tolist()
    arr_and_avg1 = avg_7d_1.tolist()
    arr_and_avg2 = avg_7d_2.tolist()
    arr_and_avg3 = avg_7d_3.tolist()
    arr_and_avg4 = avg_3d_0.tolist()
    arr_and_avg5 = avg_last7d.tolist()

    ## 外推 list
    wt = arr_list.tolist()
    func = lambda x, y: x * y
    lst = [0.2, 0.3, 0.5]

    ## 测试集数据外推 6 天
    for i in range(6):
        wt.append(sum(map(func, wt[-3:], lst)))

        arr_and_avg.append(feature_obj.clc_fc_val(pd.Series(wt[-30:])))
        arr_and_avg0.append(feature_obj.window_avg_arr(pd.Series(wt[-30:]), 0))
        arr_and_avg1.append(feature_obj.window_avg_arr(pd.Series(wt[-30:]), 1))
        arr_and_avg2.append(feature_obj.window_avg_arr(pd.Series(wt[-30:]), 2))
        arr_and_avg3.append(feature_obj.window_avg_arr(pd.Series(wt[-30:]), 3))
        arr_and_avg4.append(feature_obj.window_avg_arr(pd.Series(wt[-30:]), 4))
        arr_and_avg5.append(feature_obj.latest_val(pd.Series(wt[-30:])))
        
    try:
        sku_data = sku_data[30:]

        sku_data['avg_7d_0'] = arr_and_avg0
        sku_data['avg_7d_1'] = arr_and_avg1
        sku_data['avg_7d_2'] = arr_and_avg2
        sku_data['avg_7d_3'] = arr_and_avg3
        sku_data['roll_7d_avg'] = arr_and_avg

        sku_data['avg_3d_0'] = arr_and_avg4
        sku_data['avg_last7d'] = arr_and_avg5

    except:
        print('wh_id:{}  sku_id:{}  特征生产异常！'.format(wh_id, sku_id))
        return []
        
    result_list = []
    for val in sku_data.values:
        result_list.append(dict(zip(sku_data.keys(), val)))

    return result_list

In [17]:
data_rdd = all_data.rdd.map(lambda row:((row['bu_id'], row['wh_id'], row['sku_id']), list(row))) \
                   .aggregateByKey(list(), seqOp, combOp) \
                   .sortByKey(numPartitions=100, keyfunc=lambda x: x) \
                   .flatMap(lambda item: arr_feature(item, configs))

data_df = spark.createDataFrame(data_rdd)

In [18]:
aaa = data_df.toPandas()

In [19]:
aaa

,all_pro_num,arranged_cnt,avg_3d_0,avg_7d_0,avg_7d_1,avg_7d_2,avg_7d_3,avg_last7d,bu_id,cat1_name,...,is_holiday,is_train,is_weekend,is_work_day,pro_num,redu_num,roll_7d_avg,seq_num,sku_id,wh_id
0,0,0,2.000000,1.714286,0.571429,2.857143,2.285714,1.200000,110100,速食熟食,...,0,1,1,0,0,0,1.757143,0,10024176,159
1,0,0,1.333333,1.714286,0.571429,2.142857,2.285714,1.000000,110100,速食熟食,...,0,1,1,0,0,0,1.485714,0,10024176,159
2,0,0,0.666667,1.285714,1.000000,2.142857,1.285714,0.700000,110100,速食熟食,...,0,1,0,1,0,0,1.200000,0,10024176,159
3,0,0,0.000000,1.285714,1.000000,2.142857,0.714286,0.700000,110100,速食熟食,...,0,1,0,1,0,0,0.928571,0,10024176,159
4,0,0,0.000000,0.857143,1.142857,2.428571,0.714286,0.900000,110100,速食熟食,...,0,1,0,1,0,0,0.771429,0,10024176,159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417304,0,0,0.416667,0.750000,4.000000,1.428571,1.571429,0.742500,11000200,肉禽水产（鲜）,...,0,0,1,0,0,0,0.935714,0,10462510,362
417305,0,0,0.578333,0.676429,4.000000,1.428571,1.142857,0.873750,11000200,肉禽水产（鲜）,...,0,0,1,0,0,0,0.947857,0,10462510,362
417306,0,0,0.527500,0.754643,3.714286,1.428571,1.285714,0.664625,11000200,肉禽水产（鲜）,...,0,0,0,1,0,0,0.927500,0,10462510,362
417307,0,0,0.520583,0.544536,1.142857,4.142857,1.285714,0.532938,11000200,肉禽水产（鲜）,...,0,0,0,1,0,0,0.833821,0,10462510,362


In [20]:
## 更新配置参数字典
append_conf = {"bd_data_df_col_names": data_df.schema.names
               }
configs.update(append_conf)

In [21]:
# 销量预测函数
###############################################################################

def forecast_func(feature_rdd, configs):
    
    from sklearn import preprocessing
    from sklearn.linear_model import Ridge
    from sklearn.linear_model import LinearRegression
    
    sku_data = pd.DataFrame(feature_rdd[1], columns=configs['bd_data_df_col_names']) \
                 .replace('', np.nan)
    
    bu_id = int(sku_data.bu_id.unique()[0])
    wh_id, sku_id = int(sku_data.wh_id.unique()[0]), int(sku_data.sku_id.unique()[0])
    sku_data = sku_data.sort_values('dt')
    pred_dates = sku_data[sku_data.is_train==0].dt
    avg_result = sku_data[sku_data.is_train==0].roll_7d_avg
    
    feat_cols = ['dt', 'arranged_cnt', 'is_train', 'all_pro_num', 'seq_num',
                 'pro_num', 'redu_num', 'csu_redu_num', 'cir_redu_num', 
                 'roll_7d_avg', 'avg_7d_0', 'avg_3d_0']
    sku_data = sku_data[feat_cols]

    train_data = sku_data[sku_data.is_train==1].drop(['dt', 'is_train'], axis=1)
    test_data = sku_data[sku_data.is_train==0].drop(['dt', 'is_train'], axis=1)

    train_data_y, train_data_X = train_data.iloc[:,0], train_data.iloc[:,1:]
    test_data_y, test_data_X = test_data.iloc[:,0], test_data.iloc[:,1:]

    scaler = preprocessing.StandardScaler()
    scaler.fit(train_data_X)
    train_data_X_std = scaler.transform(train_data_X)
    test_data_X_std = scaler.transform(test_data_X)

    clf = Ridge()
    clf.fit(train_data_X_std, train_data_y)

    pred = clf.predict(test_data_X_std)
    pred = list(map(lambda x: max(x, 0.), pred))

    result = pd.DataFrame({
                           'bu_id': [bu_id]*len(pred_dates),
                           'wh_id': [wh_id]*len(pred_dates),
                           'sku_id': [sku_id]*len(pred_dates),
                           'date': pred_dates,
                           'avg_result': avg_result,
                           'predictions': pred})

    result['results'] = 0.6 * result.predictions + 0.4 * result.avg_result
    result['dt'] = configs['forecast_date']
    
    result_list = []
    for val in result.values:
        result_list.append(dict(zip(result.keys(), val)))

    return result_list

In [22]:
forecast_rdd = data_df.rdd.map(lambda row:((row['bu_id'], row['wh_id'], row['sku_id']), list(row))) \
                  .aggregateByKey(list(), seqOp, combOp) \
                  .sortByKey(numPartitions=100, keyfunc=lambda x: x) \
                  .flatMap(lambda item: forecast_func(item, configs))

forecast_df = spark.createDataFrame(forecast_rdd)

In [23]:
aaaa = forecast_df.toPandas()

In [24]:
aaaa

,avg_result,bu_id,date,dt,predictions,results,sku_id,wh_id
0,1.271429,110100,20210826,20210826,2.192767,1.824232,10024176,159
1,1.157143,110100,20210827,20210826,1.775311,1.528044,10024176,159
2,1.042857,110100,20210828,20210826,1.528095,1.334000,10024176,159
3,1.042857,110100,20210829,20210826,1.528095,1.334000,10024176,159
4,0.942857,110100,20210830,20210826,1.024500,0.991843,10024176,159
...,...,...,...,...,...,...,...,...
22542,0.935714,11000200,20210828,20210826,1.848651,1.483476,10462510,362
22543,0.947857,11000200,20210829,20210826,1.828888,1.476476,10462510,362
22544,0.927500,11000200,20210830,20210826,1.821211,1.463727,10462510,362
22545,0.833821,11000200,20210831,20210826,1.738068,1.376369,10462510,362


In [99]:
forecast_results

,bu_id,wh_id,sku_id,date,avg_result,predictions,results
0,110100,159,10024176,20210826,1.271429,2.192767,1.824232
1,110100,159,10024176,20210827,1.157143,1.775311,1.528044
2,110100,159,10024176,20210828,1.042857,1.528095,1.334000
3,110100,159,10024176,20210829,1.042857,1.528095,1.334000
4,110100,159,10024176,20210830,0.942857,1.024500,0.991843
...,...,...,...,...,...,...,...
22542,11000200,362,10462510,20210828,0.935714,1.848651,1.483476
22543,11000200,362,10462510,20210829,0.947857,1.828888,1.476476
22544,11000200,362,10462510,20210830,0.927500,1.821211,1.463727
22545,11000200,362,10462510,20210831,0.833821,1.738068,1.376369


In [80]:
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression

sku_data = pd.DataFrame(aa[2][1], columns=configs['bd_all_data_col_names']) \
             .replace('', np.nan)

bu_id = int(sku_data.bu_id.unique()[0])
wh_id, sku_id = int(sku_data.wh_id.unique()[0]), int(sku_data.sku_id.unique()[0])
sku_data = sku_data.sort_values('dt')
pred_dates = sku_data[sku_data.is_train==0].dt
avg_result = sku_data[sku_data.is_train==0].roll_7d_avg

feat_cols = ['dt', 'arranged_cnt', 'is_train', 'all_pro_num', 'seq_num',
             'pro_num', 'redu_num', 'csu_redu_num', 'cir_redu_num', 
             'roll_7d_avg', 'avg_7d_0', 'avg_3d_0']
sku_data = sku_data[feat_cols]

train_data = sku_data[sku_data.is_train==1].drop(['dt', 'is_train'], axis=1)
test_data = sku_data[sku_data.is_train==0].drop(['dt', 'is_train'], axis=1)

train_data_y, train_data_X = train_data.iloc[:,0], train_data.iloc[:,1:]
test_data_y, test_data_X = test_data.iloc[:,0], test_data.iloc[:,1:]

scaler = preprocessing.StandardScaler()
scaler.fit(train_data_X)
train_data_X_std = scaler.transform(train_data_X)
test_data_X_std = scaler.transform(test_data_X)

clf = Ridge()
clf.fit(train_data_X_std, train_data_y)

pred = clf.predict(test_data_X_std)
pred = list(map(lambda x: max(x, 0.), pred))

result = pd.DataFrame({
                       'bu_id': [bu_id]*len(pred_dates),
                       'wh_id': [wh_id]*len(pred_dates),
                       'sku_id': [sku_id]*len(pred_dates),
                       'date': pred_dates,
                       'avg_result': avg_result,
                       'predictions': pred})

result['results'] = 0.6 * result.predictions + 0.4 * result.avg_result
result['dt'] = forecast_date

In [83]:
result

,bu_id,wh_id,sku_id,date,avg_result,predictions,results,dt
152,120100,73,10057646,20210826,1.328571,2.814998,2.220427,20210826
153,120100,73,10057646,20210827,1.371429,2.729631,2.186350,20210826
154,120100,73,10057646,20210828,1.321429,2.415848,1.978080,20210826
155,120100,73,10057646,20210829,1.196429,2.487240,1.970915,20210826
156,120100,73,10057646,20210830,1.128929,2.549229,1.981109,20210826
157,120100,73,10057646,20210831,1.043393,2.624090,1.991811,20210826
158,120100,73,10057646,20210901,1.036804,2.662625,2.012297,20210826


In [89]:
wh_id, sku_id = 447, 10599580
wh_id, sku_id = 65, 10020473
# wh_id, sku_id = (100, 10541249)
wh_id, sku_id = result.wh_id.unique()[0], result.sku_id.unique()[0]
forecast_results[(forecast_results.wh_id==wh_id)
                 &(forecast_results.sku_id==sku_id)].sort_values('date')

,bu_id,wh_id,sku_id,date,avg_result,predictions,results
1148,120100,73,10057646,20210826,1.328571,2.814998,2.220427
1149,120100,73,10057646,20210827,1.371429,2.729631,2.186350
1150,120100,73,10057646,20210828,1.321429,2.415848,1.978080
1151,120100,73,10057646,20210829,1.196429,2.487240,1.970915
1152,120100,73,10057646,20210830,1.128929,2.549229,1.981109
1153,120100,73,10057646,20210831,1.043393,2.624090,1.991811
1154,120100,73,10057646,20210901,1.036804,2.662625,2.012297


In [22]:
aa = forecast_rdd.take(10)

In [23]:
aa

[((11000141, 396, 10275772),
  [[3,
    2,
    1.3333333333333333,
    1.5714285714285714,
    1.1428571428571428,
    1.5714285714285714,
    0.2857142857142857,
    1.25,
    11000141,
    '肉禽水产（鲜）',
    '鲜猪肉',
    1,
    0,
    'Mon',
    '20210719',
    0,
    1,
    0,
    1,
    0,
    1,
    1.2714285714285714,
    2,
    10275772,
    396],
   [3,
    1,
    1.6666666666666667,
    1.4285714285714286,
    1.2857142857142858,
    1.7142857142857142,
    0.42857142857142855,
    1.5499999999999998,
    11000141,
    '肉禽水产（鲜）',
    '鲜猪肉',
    1,
    0,
    'Tue',
    '20210720',
    0,
    1,
    0,
    1,
    0,
    1,
    1.3714285714285714,
    2,
    10275772,
    396],
   [4,
    0,
    1.3333333333333333,
    1.2857142857142858,
    1.5714285714285714,
    1.5714285714285714,
    0.5714285714285714,
    1.35,
    11000141,
    '肉禽水产（鲜）',
    '鲜猪肉',
    2,
    0,
    'Wed',
    '20210721',
    0,
    1,
    0,
    1,
    0,
    2,
    1.2571428571428571,
    2,
    10275772,


In [18]:
# data_df.select('bu_id', 'wh_id', 'sku_id').distinct().count()

In [47]:
df_train = train_data.toPandas()
df_test = test_data.toPandas()

In [48]:
df_all = pd.concat([df_train.drop('row_num', axis=1), df_test])

In [49]:
len(df_train), len(df_test), len(df_all)

(491392, 25648, 517040)

In [50]:
df_train.shape, df_test.shape, df_all.shape

((491392, 53), (25648, 52), (517040, 52))

In [51]:
df_all.groupby(['bu_id', 'wh_id', 'sku_id']).count()

is_train  cat1_id  cat1_name  cat2_name  two_embed0  \
bu_id    wh_id sku_id                                                          
110100   159   10000124         7        7          7          7           7   
               10024176       160      160        160        160         160   
               10028517       188      188        188        188         188   
               10030243       161      161        161        161         161   
               10033512       158      158        158        158         158   
...                           ...      ...        ...        ...         ...   
11000217 353   10283975         7        7          7          7           7   
               10331486         7        7          7          7           7   
11002954 416   10614926         7        7          7          7           7   
               10722819         7        7          7          7           7   
               10758321         7        7          7          7           7   

                         two_embed1  cnt_band  brand_name  tax_rate  \
bu_id    wh_id sku_id                                                 
110100   159   10000124           7         7           7         7   
               10024176         160       160         160        71   
               10028517         188       188         188       159   
               10030243         161       161         161       159   
               10033512         158       158         158       113   
...                             ...       ...         ...       ...   
11000217 353   10283975           7         7           7         7   
               10331486           7         7           7         7   
11002954 416   10614926           7         7           7         7   
               10722819           7         7           7         7   
               10758321           7         7           7         7   

                         csu_origin_price  ...  meet_byrs  frozen_meat_byrs  \
bu_id    wh_id sku_id                      ...                                
110100   159   10000124                 7  ...          7                 7   
               10024176                71  ...        160               160   
               10028517               159  ...        188               188   
               10030243               159  ...        161               161   
               10033512               113  ...        158               158   
...                                   ...  ...        ...               ...   
11000217 353   10283975                 7  ...          7                 7   
               10331486                 7  ...          7                 7   
11002954 416   10614926                 7  ...          7                 7   
               10722819                 7  ...          7                 7   
               10758321                 7  ...          7                 7   

                         forzen_semi_byrs  wine_byrs  rice_byrs  egg_byrs  \
bu_id    wh_id sku_id                                                       
110100   159   10000124                 7          7          7         7   
               10024176               160        160        160       160   
               10028517               188        188        188       188   
               10030243               161        161        161       161   
               10033512               158        158        158       158   
...                                   ...        ...        ...       ...   
11000217 353   10283975                 7          7          7         7   
               10331486                 7          7          7         7   
11002954 416   10614926                 7          7          7         7   
               10722819                 7          7          7         7   
               10758321                 7          7          7         7   

                         is_on

In [52]:
df_all.columns

Index(['is_train', 'bu_id', 'wh_id', 'sku_id', 'cat1_id', 'cat1_name',
       'cat2_name', 'two_embed0', 'two_embed1', 'cnt_band', 'brand_name',
       'tax_rate', 'csu_origin_price', 'w_price', 'seq_num', 'seq_price',
       'redu_num', 'csu_redu_num', 'csu_redu_mdct', 'csu_redu_adct',
       'cir_redu_num', 'cir_redu_mdct', 'cir_redu_adct', 'pro_num',
       'discount_price', 'day_abbr', 'is_work_day', 'is_weekend', 'is_holiday',
       'festival_name', 'western_festival_name', 'weather', 'weather_b',
       'avg_t', 'avg_t_b', 'new_byrs', 'old_byrs', 'poultry_byrs',
       'vege_fruit_byrs', 'egg_aquatic_byrs', 'standard_byrs', 'vege_byrs',
       'meet_byrs', 'frozen_meat_byrs', 'forzen_semi_byrs', 'wine_byrs',
       'rice_byrs', 'egg_byrs', 'is_on_shelf', 'is_outstock', 'arranged_cnt',
       'dt'],
      dtype='object')

In [53]:
# 取出需要用到的columns
data = df_all[['bu_id', 'wh_id', 'sku_id', 'cat1_name', 'cat2_name',
           'seq_num', 'redu_num', 'csu_redu_num', 'cir_redu_num', 'pro_num',
           'day_abbr', 'is_work_day', 'is_weekend', 'is_holiday', 
           'is_train', 'arranged_cnt', 'dt']]
data

,bu_id,wh_id,sku_id,cat1_name,cat2_name,seq_num,redu_num,csu_redu_num,cir_redu_num,pro_num,day_abbr,is_work_day,is_weekend,is_holiday,is_train,arranged_cnt,dt
0,110100,159,10551368,餐厨用品,一次性餐具,NaN,NaN,NaN,NaN,NaN,Fri,1,0,0,1,0,20210423
1,110100,159,10551368,餐厨用品,一次性餐具,NaN,NaN,NaN,NaN,NaN,Sun,0,1,0,1,0,20210606
2,110100,159,10551368,餐厨用品,一次性餐具,NaN,1.0,0.0,1.0,NaN,Sun,0,1,0,1,0,20210516
3,110100,159,10551368,餐厨用品,一次性餐具,0.0,NaN,NaN,NaN,2.0,Sun,0,1,0,1,1,20210808
4,110100,159,10551368,餐厨用品,一次性餐具,0.0,NaN,NaN,NaN,2.0,Wed,1,0,0,1,2,20210804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25643,11000047,139,10443438,蔬菜水果,新鲜蔬菜,NaN,2.0,0.0,2.0,NaN,Sun,0,1,0,0,0,20210829
25644,11000047,139,10443438,蔬菜水果,新鲜蔬菜,NaN,1.0,0.0,1.0,NaN,Mon,1,0,0,0,0,20210830
25645,11000047,139,10443438,蔬菜水果,新鲜蔬菜,NaN,NaN,NaN,NaN,NaN,Wed,1,0,0,0,0,20210901
25646,11000047,139,10443438,蔬菜水果,新鲜蔬菜,NaN,2.0,0.0,2.0,NaN,Fri,1,0,0,0,0,20210827


In [54]:
data['dt'] = data['dt'].astype('str')

In [55]:
# 填充缺失值
data = data.fillna(0)
data.isna().sum()

bu_id           0
wh_id           0
sku_id          0
cat1_name       0
cat2_name       0
seq_num         0
redu_num        0
csu_redu_num    0
cir_redu_num    0
pro_num         0
day_abbr        0
is_work_day     0
is_weekend      0
is_holiday      0
is_train        0
arranged_cnt    0
dt              0
dtype: int64

In [56]:
# 整体促销数量
data['all_pro_num'] = data['seq_num'] + data['redu_num'] + data['pro_num']
data = data.sort_values('dt')
data

,bu_id,wh_id,sku_id,cat1_name,cat2_name,seq_num,redu_num,csu_redu_num,cir_redu_num,pro_num,day_abbr,is_work_day,is_weekend,is_holiday,is_train,arranged_cnt,dt,all_pro_num
126381,11000039,136,10057762,冷冻饮品、饮料、酒,饮料类,0.0,0.0,0.0,0.0,0.0,Thu,1,0,0,1,0,20210225,0.0
240023,110100,316,10073132,调料干货,火锅汤料,0.0,1.0,0.0,1.0,0.0,Thu,1,0,0,1,15,20210225,1.0
310983,310100,332,10038053,冷冻饮品、饮料、酒,饮料类,0.0,0.0,0.0,0.0,0.0,Thu,1,0,0,1,1,20210225,0.0
430297,11000008,93,10253502,蔬菜水果,新鲜蔬菜,0.0,0.0,0.0,0.0,0.0,Thu,1,0,0,1,6,20210225,0.0
141582,110100,316,10279151,餐厨用品,一次性餐具,0.0,1.0,0.0,1.0,0.0,Thu,1,0,0,1,0,20210225,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23025,11000039,136,10407987,调理包,素菜料包,0.0,0.0,0.0,0.0,2.0,Wed,1,0,0,0,0,20210901,2.0
13355,11000033,111,10050842,调料干货,调味酱,0.0,0.0,0.0,0.0,0.0,Wed,1,0,0,0,0,20210901,0.0
6810,340100,85,10494781,冷冻半成品,鸡肉调理品,0.0,0.0,0.0,0.0,0.0,Wed,1,0,0,0,0,20210901,0.0
19644,310100,332,10831035,调料干货,调味汁,0.0,0.0,0.0,0.0,0.0,Wed,1,0,0,0,0,20210901,0.0


In [57]:
def clc_fc_val(list_actu_T0):
    list_actu_T0 = list_actu_T0.tolist()
    w_1_3 = 0.3
    w_1_7 = 0.3
    w_8_14 = 0.2
    w_15_21 = 0.1
    w_22_28 = 0.1
    # 删除最大值和最小值
    if len(list_actu_T0) ==0 :
        return 0.
    list_actu_T0.remove(max(list_actu_T0))
    if len(list_actu_T0) ==0 :
        return 0.
    list_actu_T0.remove(min(list_actu_T0))
    if len(list_actu_T0) ==0 :
        return 0.
    # list 前闭后开
    avg_1_3 =   np.nan_to_num(np.mean(list_actu_T0[-3:])       )
    avg_1_7 =   np.nan_to_num(np.mean(list_actu_T0[-7:])       )
    avg_8_14 =  np.nan_to_num(np.mean(list_actu_T0[-14:-7])    )
    avg_15_21 = np.nan_to_num(np.mean(list_actu_T0[-21:-14])   )
    avg_22_28 = np.nan_to_num(np.mean(list_actu_T0[-28:-21])   )
    return float(avg_1_7 * w_1_7 + avg_8_14 * w_8_14 + avg_15_21 * w_15_21 + avg_22_28 * w_22_28 + avg_1_3 * w_1_3)

In [58]:
def window_avg_arr(list_actu_T0, cnt):
    list_actu_T0 = list_actu_T0.tolist()
    if len(list_actu_T0) ==0 :
        return 0.
    
#     list_actu_T0.remove(max(list_actu_T0))
#     if len(list_actu_T0) ==0 :
#         return 0.
#     list_actu_T0.remove(min(list_actu_T0))
#     if len(list_actu_T0) ==0 :
#         return 0.
    
    if cnt == 0:
        avg_arr = np.nan_to_num(np.mean(list_actu_T0[-7:]))
    elif cnt == 4:
        avg_arr = np.nan_to_num(np.mean(list_actu_T0[-3:]))
    else:
        avg_arr = np.nan_to_num(np.mean(list_actu_T0[-7*(cnt+1):-7*cnt]))
    return avg_arr

In [59]:
def latest_val(list_actu_T0):
    list_actu_T0 = list_actu_T0.tolist()
    w_1_3  = 0.5
    w_4_10 = 0.5
    
    val_1_3  = list_actu_T0[-3:]
    val_4_10 = list_actu_T0[-10:-4] 
    # 删除最大值
    if len(val_1_3) * len(val_4_10) ==0 :
        return 0.
    val_4_10.remove(max(val_4_10))
    if len(val_4_10) ==0 :
        return 0.
    
    func = lambda x, y: x * y
    lst = [0.2, 0.3, 0.5]

    # list 前闭后开
    avg_1_3  = np.nan_to_num(sum(map(func, val_1_3, lst)))
#     avg_1_3  = np.nan_to_num(np.mean(val_1_3) )
    avg_4_10 = np.nan_to_num(np.mean(val_4_10))

    return float(avg_1_3 * w_1_3 + avg_4_10 * w_4_10)

In [60]:
import tqdm

In [61]:
%%time

sku_data_list = []
for (wh_id, sku_id), sku_data in tqdm.tqdm(data.groupby(['wh_id', 'sku_id'])):
    if len(sku_data[sku_data.is_train==1]) < 60:
        #or len(sku_data[sku_data.is_train==0]) < 7 :
        continue
    
    arr_list = sku_data[sku_data.is_train == 1].arranged_cnt
    
    roll_7d_avg = arr_list.rolling(30, min_periods=30).apply(clc_fc_val).dropna()

    avg_7d_0 = arr_list.rolling(30, min_periods=30).apply(lambda x: window_avg_arr(x, 0)).dropna()
    avg_7d_1 = arr_list.rolling(30, min_periods=30).apply(lambda x: window_avg_arr(x, 1)).dropna()
    avg_7d_2 = arr_list.rolling(30, min_periods=30).apply(lambda x: window_avg_arr(x, 2)).dropna()
    avg_7d_3 = arr_list.rolling(30, min_periods=30).apply(lambda x: window_avg_arr(x, 3)).dropna()

    avg_3d_0 = arr_list.rolling(30, min_periods=30).apply(lambda x: window_avg_arr(x, 4)).dropna()
    avg_last7d = arr_list.rolling(30, min_periods=30).apply(lambda x: latest_val(x)).dropna()
    
    
    # 另一种外推方法 
    arr_and_avg  = roll_7d_avg.tolist()
    arr_and_avg0 = avg_7d_0.tolist()
    arr_and_avg1 = avg_7d_1.tolist()
    arr_and_avg2 = avg_7d_2.tolist()
    arr_and_avg3 = avg_7d_3.tolist()
    arr_and_avg4 = avg_3d_0.tolist()
    arr_and_avg5 = avg_last7d.tolist()

#     arr_and_avg_ = roll_7d_avg.tolist()

    wt = arr_list.tolist()
    func = lambda x, y: x * y
    lst = [0.2, 0.3, 0.5]

    for i in range(6):
    #     wt.append(clc_fc_val(pd.Series(wt[-30:])))
        wt.append(sum(map(func, wt[-3:], lst)))

        arr_and_avg.append(clc_fc_val(pd.Series(wt[-30:])))
        arr_and_avg0.append(window_avg_arr(pd.Series(wt[-30:]), 0))
        arr_and_avg1.append(window_avg_arr(pd.Series(wt[-30:]), 1))
        arr_and_avg2.append(window_avg_arr(pd.Series(wt[-30:]), 2))
        arr_and_avg3.append(window_avg_arr(pd.Series(wt[-30:]), 3))
        arr_and_avg4.append(window_avg_arr(pd.Series(wt[-30:]), 4))
        arr_and_avg5.append(latest_val(pd.Series(wt[-30:])))

#         arr_and_avg_.append(np.mean(arr_and_avg_[-3:]))

        
    try:
#         sku_data1 = sku_data.copy()
        sku_data = sku_data[30:]


        sku_data['avg_7d_0'] = arr_and_avg0
        sku_data['avg_7d_1'] = arr_and_avg1
        sku_data['avg_7d_2'] = arr_and_avg2
        sku_data['avg_7d_3'] = arr_and_avg3
        sku_data['roll_7d_avg'] = arr_and_avg

        sku_data['avg_3d_0'] = arr_and_avg4
        sku_data['avg_last7d'] = arr_and_avg5
        
#         sku_data['avg_result'] = arr_and_avg_

        if sku_data['bu_id'].isna().sum() != 0:
            break
        sku_data_list.append(sku_data)
    except:
        print((wh_id, sku_id))

100%|██████████| 3664/3664 [11:59<00:00,  5.09it/s]

CPU times: user 11min 55s, sys: 2.9 s, total: 11min 57s
Wall time: 12min 1s


In [62]:
x = pd.concat(sku_data_list).reset_index(drop=True)
x

,bu_id,wh_id,sku_id,cat1_name,cat2_name,seq_num,redu_num,csu_redu_num,cir_redu_num,pro_num,...,arranged_cnt,dt,all_pro_num,avg_7d_0,avg_7d_1,avg_7d_2,avg_7d_3,roll_7d_avg,avg_3d_0,avg_last7d
0,310100,65,10020473,调料干货,调味汁,0.0,0.0,0.0,0.0,0.0,...,0,20210327,0.0,0.714286,2.285714,1.571429,0.000000,0.928571,0.333333,0.800000
1,310100,65,10020473,调料干货,调味汁,0.0,3.0,0.0,3.0,0.0,...,2,20210328,3.0,0.714286,2.000000,1.857143,0.000000,0.800000,0.000000,0.400000
2,310100,65,10020473,调料干货,调味汁,0.0,2.0,0.0,2.0,0.0,...,0,20210329,2.0,1.000000,1.714286,2.142857,0.000000,0.957143,0.666667,1.000000
3,310100,65,10020473,调料干货,调味汁,0.0,3.0,0.0,3.0,0.0,...,2,20210330,3.0,0.714286,2.000000,1.142857,1.000000,0.928571,0.666667,0.600000
4,310100,65,10020473,调料干货,调味汁,0.0,2.0,0.0,2.0,0.0,...,1,20210331,2.0,0.714286,2.285714,0.571429,1.571429,1.185714,1.333333,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417304,310100,447,10599580,米面粮油,米,0.0,5.0,0.0,5.0,0.0,...,0,20210828,5.0,0.435714,1.857143,1.428571,2.857143,1.192857,0.683333,0.412500
417305,310100,447,10599580,米面粮油,米,0.0,5.0,0.0,5.0,1.0,...,0,20210829,6.0,0.382143,2.000000,1.285714,2.857143,1.025000,0.558333,0.388750
417306,310100,447,10599580,米面粮油,米,0.0,5.0,0.0,5.0,1.0,...,0,20210830,6.0,0.464643,2.000000,1.142857,2.857143,1.000357,0.584167,0.493125
417307,310100,447,10599580,米面粮油,米,0.0,5.0,0.0,5.0,1.0,...,0,20210831,6.0,0.548393,1.714286,1.142857,2.714286,0.986250,0.596250,0.445688


In [63]:
# wh_id, sku_id = 447, 10599580
# wh_id, sku_id = 65, 10020473
# # wh_id, sku_id = (100, 10541249)
# # wh_id, sku_id = 159, 10000124
# qq = aaa[(aaa.wh_id==wh_id)
#  &(aaa.sku_id==sku_id)].sort_values('dt').roll_7d_avg.tolist() 
# pp = x[(x.wh_id==wh_id)
#  &(x.sku_id==sku_id)].sort_values('dt').roll_7d_avg.tolist()

# list(zip(pp,qq))

# pp == qq

In [64]:
data = x.copy()
data.columns

Index(['bu_id', 'wh_id', 'sku_id', 'cat1_name', 'cat2_name', 'seq_num',
       'redu_num', 'csu_redu_num', 'cir_redu_num', 'pro_num', 'day_abbr',
       'is_work_day', 'is_weekend', 'is_holiday', 'is_train', 'arranged_cnt',
       'dt', 'all_pro_num', 'avg_7d_0', 'avg_7d_1', 'avg_7d_2', 'avg_7d_3',
       'roll_7d_avg', 'avg_3d_0', 'avg_last7d'],
      dtype='object')

In [65]:
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression

forecast_results = []

for (bu_id, wh_id, sku_id), sku_data in tqdm.tqdm(data.groupby(['bu_id', 'wh_id','sku_id'])):
    sku_data = sku_data.sort_values('dt')
    pred_dates = sku_data[sku_data.is_train==0].dt
    avg_result = sku_data[sku_data.is_train==0].roll_7d_avg
    
    sku_data = sku_data[['dt', 'arranged_cnt', 'is_train', 'all_pro_num', 
                         'seq_num', 'pro_num', 'redu_num', 'csu_redu_num', 'cir_redu_num', 
                         'roll_7d_avg', 'avg_7d_0', 'avg_3d_0']]
    
#     sku_data = sku_data[['dt', 'arranged_cnt', 'is_train', 'all_pro_num', 
#                          'seq_num', 'pro_num', 'redu_num', 'csu_redu_num', 'cir_redu_num', 
#                          'roll_7d_avg', 'avg_7d_0', 'avg_3d_0',
#                          'Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed']]

    train_data = sku_data[sku_data.is_train==1].drop(['dt', 'is_train'], axis=1)
    test_data = sku_data[sku_data.is_train==0].drop(['dt', 'is_train'], axis=1)

    train_data_y, train_data_X = train_data.iloc[:,0], train_data.iloc[:,1:]
    test_data_y, test_data_X = test_data.iloc[:,0], test_data.iloc[:,1:]
    
    scaler = preprocessing.StandardScaler()
    scaler.fit(train_data_X)
    train_data_X_std = scaler.transform(train_data_X)
    test_data_X_std = scaler.transform(test_data_X)
    
    clf = Ridge()
    clf.fit(train_data_X_std, train_data_y)
    
    pred = clf.predict(test_data_X_std)
    pred = list(map(lambda x: max(x, 0.), pred))
    
    result = pd.DataFrame({
                           'bu_id': [bu_id]*len(pred_dates),
                           'wh_id': [wh_id]*len(pred_dates),
                           'sku_id': [sku_id]*len(pred_dates),
                           'date': pred_dates,
                           'avg_result': avg_result,
                           'predictions': pred})
    
    result['results'] = 0.6 * result.predictions + 0.4 * result.avg_result
#     result['results'] = 0.5 * result.predictions + 0.5 * result.avg_result
#     result['results'] = 0.4 * result.predictions + 0.6 * result.avg_result
    
    forecast_results.append(result)

100%|██████████| 3221/3221 [00:36<00:00, 88.23it/s]


In [66]:
forecast_results = pd.concat(forecast_results).reset_index(drop=True)
forecast_results

,bu_id,wh_id,sku_id,date,avg_result,predictions,results
0,110100,159,10024176,20210826,1.271429,2.192767,1.824232
1,110100,159,10024176,20210827,1.157143,1.775311,1.528044
2,110100,159,10024176,20210828,1.042857,1.528095,1.334000
3,110100,159,10024176,20210829,1.042857,1.528095,1.334000
4,110100,159,10024176,20210830,0.942857,1.024500,0.991843
...,...,...,...,...,...,...,...
22542,11000200,362,10462510,20210828,0.935714,1.848651,1.483476
22543,11000200,362,10462510,20210829,0.947857,1.828888,1.476476
22544,11000200,362,10462510,20210830,0.927500,1.821211,1.463727
22545,11000200,362,10462510,20210831,0.833821,1.738068,1.376369


In [67]:
forecast_results.groupby(['bu_id', 'wh_id', 'sku_id']).count()

date  avg_result  predictions  results
bu_id    wh_id sku_id                                          
110100   159   10024176     7           7            7        7
               10028517     7           7            7        7
               10030243     7           7            7        7
               10033512     7           7            7        7
               10033549     7           7            7        7
...                       ...         ...          ...      ...
11000200 362   10063316     7           7            7        7
               10063432     7           7            7        7
               10063885     7           7            7        7
               10271031     7           7            7        7
               10462510     7           7            7        7

[3221 rows x 4 columns]

## T7MAPE